In [ ]:
# 常用api测试

In [1]:
import tensorflow as tf 
from tensorflow.python.ops import nn 

def pairwise_cross_entropy_v2(
    session_id, labels, logits,
    weights=1.0, use_weight=False,
    margin=10.0, use_margin=True,
    add_batchin_neg=False, sample_mask=None, ts=None):
    if sample_mask is not None:
        session_id = tf.reshape(tf.boolean_mask(session_id, sample_mask), [-1, 1])
        labels = tf.reshape(tf.boolean_mask(labels, sample_mask), [-1, 1])
        logits = tf.reshape(tf.boolean_mask(logits, sample_mask), [-1, 1])
        weights = tf.reshape(tf.boolean_mask(weights, sample_mask), [-1, 1])

    session_mask = tf.cast(tf.equal(session_id, tf.transpose(session_id)), tf.float32)
    session_mask = session_mask - tf.matrix_diag(tf.diag_part(session_mask))

    if add_batchin_neg:
        session_outer_mask = tf.transpose(tf.random_shuffle(tf.transpose(session_mask)))
        session_mask = session_mask + session_outer_mask
        session_mask = session_mask - tf.matrix_diag(tf.diag_part(session_mask))

    labels_mask = tf.greater(labels - tf.transpose(labels), 0)
    ts_mask = tf.greater(ts - tf.transpose(ts), 0)
    labels_mask = tf.logical_or(labels_mask, ts_mask) 
    labels_mask = tf.cast(labels_mask, tf.float32)
    final_mask = tf.multiply(session_mask, labels_mask)

    logits_mat = logits - tf.transpose(logits)
    if use_margin:
        logits_mask = tf.cast(tf.less(logits_mat, margin), tf.float32)
        final_mask = tf.multiply(final_mask, logits_mask)

    final_mask = tf.cast(final_mask, tf.bool)
    logits_mat_valid = tf.boolean_mask(logits_mat, final_mask)

    if use_weight:
        weights_mat = tf.abs(weights - tf.transpose(weights))
        weights = tf.boolean_mask(weights_mat, final_mask)
    else:
        weights = tf.ones_like(logits_mat_valid)

    labels_mat_psudo = tf.ones_like(logits_mat_valid)

    loss = tf.losses.sigmoid_cross_entropy(
        multi_class_labels=labels_mat_psudo, logits=logits_mat_valid, weights=weights)
    loss = tf.where(tf.is_nan(loss), tf.zeros_like(loss), loss)

    return loss


In [ ]:
def weighted_gauc_loss(utdid_tensor, ts_tensor, label_tensor, logit_tensor, use_weight=False, alpha=1.0,
                       epsilon=1E-6):
    utdid_mask = tf.cast(tf.equal(utdid_tensor, tf.transpose(utdid_tensor)), tf.float32)
    utdid_mask = utdid_mask - tf.matrix_diag(tf.diag_part(utdid_mask))

    label_mask = 1 - tf.cast(tf.equal(label_tensor, tf.transpose(label_tensor)), tf.float32)
    label_mask = label_mask - tf.matrix_diag(tf.diag_part(label_mask))

    final_mask = tf.multiply(utdid_mask, label_mask)

    prob_mat = tf.sigmoid(logit_tensor - tf.transpose(logit_tensor))
    ts_mat = alpha * tf.abs(ts_tensor - tf.transpose(ts_tensor)) + 1.0
    label_mat = (label_tensor - tf.transpose(label_tensor) + 1.0) / 2.0

    final_weight = final_mask

    ent_mat = -tf.multiply(label_mat, tf.log(prob_mat + epsilon)) - tf.multiply(
        (1 - label_mat), tf.log(1 - prob_mat + epsilon)) # 手写 二分类的celoss
    if use_weight:
        final_weight = tf.multiply(final_mask, ts_mat)

    return tf.reduce_sum(tf.multiply(ent_mat, final_weight)) / (tf.reduce_sum(final_mask) + epsilon) # 对多个样本的celoss做reduce
